In [1]:
import requests
from collections import Counter
import pandas as pd

I changed the start period to 2018 so it was faster to load

In [2]:
r = requests.get("http://stat.data.abs.gov.au/sdmx-json/data/LF/0+1+2+3+4+5+6+7+8.1+2+3+4+5+11+12+13+7+8+6+9+16+15+14+17+10+18+19+20.1+3+2.1599+1519.10+20+30.M/all?detail=Full&dimensionAtObservation=AllDimensions&startPeriod=2018")

In [3]:
r.raise_for_status()

In [4]:
df = pd.DataFrame(r.json()["dataSets"][0]["observations"])

In [5]:
df = df.T.reset_index()

In [6]:
df = df.rename(
    columns={
        0: "observation_0", 
        1: "observation_1", 
        2: "observation_2"
    }
)

In [7]:
df_index = df["index"].str.split(":", expand=True)
del df["index"]

The values in the colon delimited string and the id values in the structure section at the end of the json dont seem to match. I made the assumtion that the order is preseved in the structure relative to the values in the colon delimited string. This may be incorrect!

It's worth validating this data somehow rather than just assuming this worked. If it is incorrect, I placed the logic in a function so it should be easy to modify

In [8]:
def map_column(df_index, data, column):
    """
    map integers in df_index to their corresponding names provided in the
    structure section at the end of the json
    """
    structure = data["structure"]["dimensions"]["observation"]
    keys = Counter(df_index[column]).keys() #counter not actually needed here
    values = [x["name"] for x in structure[column]["values"]]
    mapping = dict(zip(keys, values))
    return df_index[column].map(mapping)

In [9]:
# use map column on each of the columns in df_index
for column in df_index:
    df_index[column] = map_column(df_index, data=r.json(), column=column)

In [10]:
# rename the columns in df_index based on the names provided in structure
structure = r.json()["structure"]["dimensions"]["observation"]
new_column_names = [x["name"] for x in structure]
old_column_names = df_index.columns

df_index = df_index.rename(
    columns=dict(zip(old_column_names, new_column_names))
)

In [11]:
df_index.head()

,Region,Data Item,Sex,Age,Adjustment Type,Frequency,Time
0,Australia,Civilian population ('000),Males,15 and over,Original,Monthly,Jan-2018
1,Australia,Civilian population ('000),Males,15 and over,Original,Monthly,Feb-2018
2,Australia,Civilian population ('000),Males,15 and over,Original,Monthly,Mar-2018
3,Australia,Civilian population ('000),Males,15 and over,Original,Monthly,Apr-2018
4,Australia,Civilian population ('000),Males,15 and over,Original,Monthly,May-2018


In [12]:
# join index back to the df with the observations
df = df.join(df_index)

In [13]:
df.head()

,observation_0,observation_1,observation_2,Region,Data Item,Sex,Age,Adjustment Type,Frequency,Time
0,9859.155,0.0,NaN,Australia,Civilian population ('000),Males,15 and over,Original,Monthly,Jan-2018
1,9877.152,0.0,NaN,Australia,Civilian population ('000),Males,15 and over,Original,Monthly,Feb-2018
2,9895.154,0.0,NaN,Australia,Civilian population ('000),Males,15 and over,Original,Monthly,Mar-2018
3,9907.386,0.0,NaN,Australia,Civilian population ('000),Males,15 and over,Original,Monthly,Apr-2018
4,9919.636,0.0,NaN,Australia,Civilian population ('000),Males,15 and over,Original,Monthly,May-2018
